In [1]:
# Email: SOTF Zip Code Request from Faith
# 12:05 am 20200312

import pandas as pd
import numpy as np
import datetime
import os
import json
from haversine import haversine

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json"))

print(datetime.datetime.now())

2020-03-12 16:28:29.412075


In [2]:
df_SOTF=pd.read_excel("./Big Lots SOTF GOs Addresses with Store # - 3.12.20.xlsx",
                      sheet_name="Sheet1",dtype=str,skiprows=1)
df_SOTF=df_SOTF.rename(columns={'Store #':"location_id"})
df_SOTF=df_SOTF[df_SOTF['location_id'].apply(type)==str]
df_SOTF=df_SOTF[df_SOTF['location_id'].str.isdigit()]

df_SOTF.shape,df_SOTF['location_id'].nunique()


((126, 4), 126)

In [3]:
df_SOTF.groupby("location_id")['City'].count().to_frame().reset_index().sort_values("City",ascending=False).head(2)

,location_id,City
0,1012,1
79,5153,1


In [4]:
df_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200301-134228-900.txt",
                           dtype=str,sep="|",usecols=['location_id','zip_cd','latitude_meas','longitude_meas'])
df_store_list['zip_cd']=df_store_list['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
df_store_list['latitude_meas']=df_store_list['latitude_meas'].astype(float)
df_store_list['longitude_meas']=df_store_list['longitude_meas'].astype(float)

df_store_list=df_store_list[['location_id','zip_cd','latitude_meas','longitude_meas']]
df_store_list=df_store_list.rename(columns={"zip_cd":"store_zip"})

df_SOTF=pd.merge(df_SOTF,df_store_list,on="location_id",how="left")
df_SOTF.shape

(126, 7)

In [5]:
df_new_stores=df_SOTF[pd.isnull(df_SOTF['latitude_meas'])]
df_new_stores=df_new_stores.sort_values("location_id")
df_new_stores.shape

(24, 7)

In [6]:
new_store_list=df_new_stores['location_id'].unique().tolist()
new_store_list

['4710',
 '4712',
 '4713',
 '4714',
 '4715',
 '4716',
 '4717',
 '4718',
 '4719',
 '4720',
 '4721',
 '4722',
 '5405',
 '5412',
 '5413',
 '5415',
 '5416',
 '5417',
 '5418',
 '5419',
 '5420',
 '5421',
 '5422',
 '5423']

In [7]:
list_store_not_found=['4721','5421'] # received in email from Margaret
# RE: 2020 SOTF/Grand Openings - Local TV/Radio/Remotes Estimated Cost Request
# 20200218 at 12:52 PM

dict_new_stores={}

# 4710 	255 WEDDINGTON BRANCH ROAD	MAGIC MART SC	PIKEVILLE	KY	41501-3205
dict_new_stores.update({"4710":{"Google_Address":"255 Weddington Branch Rd, Pikeville, KY 41501","Google_Location":[37.5257758,-82.5685399]}})

# 4712 	2685 HILLTOP DRIVE	KMART SC	REDDING EPA ID, BAG BAN	CA	96002-1020
dict_new_stores.update({"4712":{"Google_Address":"2685 Hilltop Dr, Redding, CA 96002","Google_Location":[40.5684416,-122.357451]}})

# 4713 783 DELSEA DRIVE	COLLEGETOWN SHOPPING CENTER	GLASSBORO	NJ	08028
dict_new_stores.update({"4713":{"Google_Address":"731 Delsea Dr, Glassboro, NJ 08028","Google_Location":[39.72152,-75.116531]}})

# 4714 1251 STATE ROUTE 29	KMART PLAZA	GREENWICH, BAG BAN	NY	12834-6119
dict_new_stores.update({"4714":{"Google_Address":"1251 NY-29, Greenwich, NY 12834","Google_Location":[43.0973489,-73.5079692]}})

# 4715 4525 N ORACLE ROAD	TOYS R US CENTER	TUSCON	AZ	85705-1637
dict_new_stores.update({"4715":{"Google_Address":"4525 N Oracle Rd, Tucson, AZ 85705","Google_Location":[32.2885652,-110.9800392]}})

# 4716 4208 E BLUE GRASS RD	INDIAN HILLS PLAZA	MOUNT PLEASANT	MI	48858-7966
dict_new_stores.update({"4716":{"Google_Address":"4208 E Blue Grass Rd, Mt Pleasant, MI 48858","Google_Location":[43.5739459,-84.7649652]}})

# 4717 360 EAST 10TH ST	GILROY VILLAGE	GILROY EPA ID, BAG BAN	CA	95020-6576
dict_new_stores.update({"4717":{"Google_Address":"360 E 10th St, Gilroy, CA 95020","Google_Location":[37.0000558,-121.561048]}})

# 4718 1631 WEST CRAIG ROAD	NORTH MESA PLAZA	NORTH LAS VEGAS	NV	89032
dict_new_stores.update({"4718":{"Google_Address":"1631 W Craig Rd, North Las Vegas, NV 89030","Google_Location":[36.2381894,-115.1644729]}})

# 4719 3779 ROME ROAD	PULASKI PLAZA	PULASKI  BAG BAN	NY	13142-2402
dict_new_stores.update({"4719":{"Google_Address":"3779 NY-13 B, Pulaski, NY 13142","Google_Location":[43.5551558,-76.1163317]}})

# 4720 2700 US HIGHWAY 22	UNION 22 PLAZA	UNION	NJ	07083-8564
dict_new_stores.update({"4720":{"Google_Address":"2700 US-22, Union, NJ 07083","Google_Location":[40.6866464,-74.311087]}})

# 4721 400 Main Street, Dennis Port, MA, 02639
dict_new_stores.update({"4721":{"Google_Address":"400 Main St, Dennis Port, MA 02639","Google_Location":[41.66689,-70.1329873]}})

# 5405 1300 US HIGHWAY 127	FRANKLIN SQUARE & FRANKLIN WEST SHOPPING CENTER	FRANKFORT	KY	40601-4395
dict_new_stores.update({"5405":{"Google_Address":"1300 US-127, Frankfort, KY 40601","Google_Location":[38.166582,-84.8998445]}})

# 5412 2120 EAST DOROTHY LANE	KETTERING TOWN CENTER	KETTERING	OH	45420
dict_new_stores.update({"5412":{"Google_Address":"2120 E Dorothy Ln, Kettering, OH 45420","Google_Location":[39.701601,-84.1264112]}})

# 5413 340 SEVILLE ST	KOHLS SC	FLORENCE	AL	35634-1511
dict_new_stores.update({"5413":{"Google_Address":"350 Seville St, Florence, AL 35630","Google_Location":[34.8382162,-87.6269289]}})

# 5415 7743 SUDLEY ROAD	SUDLEY TOWNE PLAZA	MANASSAS	VA	20109-2813
dict_new_stores.update({"5415":{"Google_Address":"7665 Sudley Rd, Manassas, VA 20109","Google_Location":[38.7900885,-77.5121781]}})

# 5417 558 BATTLEFIELD PARKWAY	FORT OGLETHORPE MARKETPLACE	FORT OGLERTHORPE	GA	30742-3849
dict_new_stores.update({"5417":{"Google_Address":"558 Battlefield Pkwy, Fort Oglethorpe, GA 30742","Google_Location":[34.9547068,-85.2557537]}})

# 5418 1550 COSHOCTON AVE	KNOX VILLAGE SQUARE	MOUNT VERNON	OH	43050-1475
dict_new_stores.update({"5418":{"Google_Address":"1500 Coshocton Ave (US Highway 36, Mt Vernon, OH 43050","Google_Location":[40.4001295,-82.4460567]}})

# 5419 1077 EAST STUART DRIVE SUITE 100	TWIN COUNTRY PLAZA	GALAX	VA	24333-2638
dict_new_stores.update({"5419":{"Google_Address":"1077 E Stuart Dr #100, Galax, VA 24333","Google_Location":[36.6845984,-80.8871837]}})

# 5420 4025 POPLAR LEVEL ROAD	KMART CENTER	LOUISVILLE	KY	40213-1523
dict_new_stores.update({"5420":{"Google_Address":"4025 Poplar Level Rd, Louisville, KY 40213","Google_Location":[38.2009162,-85.7071697]}})

# 5421 736 Warrenton Road, Fredericksburg, VA, 22406
dict_new_stores.update({"5421":{"Google_Address":"736 Warrenton Rd, Fredericksburg, VA 22406","Google_Location":[38.3531101,-77.5043113]}})

# 4722 
dict_new_stores.update({"4722":{"Google_Address":"3820 W Ina Rd, Tucson, AZ 85741","Google_Location":[32.3387262,-111.0482994]}})

# 5416 
dict_new_stores.update({"5416":{"Google_Address":"260 Remount Rd, Front Royal, VA 22630","Google_Location":[38.9090477,-78.1831537]}})

# 5422 
dict_new_stores.update({"5422":{"Google_Address":"433 Columbia Ave, Lexington, SC 29072","Google_Location":[33.9881779,-81.2408026]}})

# 5423 
dict_new_stores.update({"5423":{"Google_Address":"1806 N Jackson St, Tullahoma, TN 37388","Google_Location":[35.3838111,-86.230163]}})


In [8]:
keys=list(dict_new_stores.keys())

In [9]:
[x for x in new_store_list if x not in keys]

[]

In [10]:
[x for x in keys if x not in new_store_list]

[]

In [11]:
df_SOTF_existing=df_SOTF[pd.notnull(df_SOTF['latitude_meas'])]
df_SOTF_new=df_SOTF[pd.isnull(df_SOTF['latitude_meas'])]
df_SOTF_new['store_zip']=df_SOTF_new['location_id'].apply(lambda x: dict_new_stores[x]['Google_Address'][-5:])
df_SOTF_new['latitude_meas']=df_SOTF_new['location_id'].apply(lambda x: dict_new_stores[x]['Google_Location'][0])
df_SOTF_new['longitude_meas']=df_SOTF_new['location_id'].apply(lambda x: dict_new_stores[x]['Google_Location'][1])

df_SOTF=df_SOTF_existing.append(df_SOTF_new)
df_SOTF=df_SOTF.sort_values("location_id")

/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [12]:
TA_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
TA_zips=TA_zips.parse("view_by_store",dtype=str)
TA_zips.head(2)

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,NaN,75040,GARLAND,TX,32.945015,-96.619194,{'DALLAS-FT. WORTH'},{'DALLAS'},"['75040', '75044', '75048', '75098']","['75089', '75042', '75043', '75041', '75088', ...",[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,NaN,75160,TERRELL,TX,32.738772,-96.299877,{'DALLAS-FT. WORTH'},"{'HUNT', 'KAUFMAN', 'ROCKWALL'}","['75160', '75126', '75161']","['75142', '75169', '75474', '75143', '75114', ...",[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."


In [13]:
df_temporary=TA_zips[['location_id','trans_P_zips_70_within_TA','trans_S_zips_70_within_TA','zips_in_10']]
df_zip_by_store=pd.DataFrame()

for ind,row in df_temporary.iterrows():
    location_id=row['location_id']
    P_zips=eval(row['trans_P_zips_70_within_TA'])
    S_zips=eval(row['trans_S_zips_70_within_TA'])
    zip_10=eval(row['zips_in_10'])
    

    df_P=pd.DataFrame(zip([location_id]*len(P_zips),P_zips))
    if len(df_P)>0:
        df_P.columns=['location_id','zip_cd']
        df_P['zip_type']="P"
        
    df_S=pd.DataFrame(zip([location_id]*len(S_zips),S_zips))
    if len(df_S)>0:
        df_S.columns=['location_id','zip_cd']
        df_S['zip_type']="S"
    
    df_10=pd.DataFrame(zip([location_id]*len(zip_10),zip_10))
    if len(df_10)>0:
        df_10.columns=['location_id','zip_cd']
        df_10['zip_type']="zip_10"
    
    df_zip_by_store=df_zip_by_store.append(df_P).append(df_S).append(df_10)

In [14]:
df_SOTF_output=pd.merge(df_SOTF,df_zip_by_store,on="location_id",how="left")
df_SOTF_output.head(2)

,Reference #,City,Address,location_id,store_zip,latitude_meas,longitude_meas,zip_cd,zip_type
0,60,Columbia,6880 Garners Ferry Road,1012,29209,33.97687,-80.956568,29223,P
1,60,Columbia,6880 Garners Ferry Road,1012,29209,33.97687,-80.956568,29205,P


In [15]:
df_SOTF_output_2=df_SOTF_output[pd.isnull(df_SOTF_output['zip_cd'])]
df_SOTF_output_1=df_SOTF_output[pd.notnull(df_SOTF_output['zip_cd'])]
print(df_SOTF_output_2.shape)

del df_SOTF_output_2['zip_cd']
del df_SOTF_output_2['zip_type']

(25, 9)


In [16]:
df_10_zips_all_new_stores=pd.DataFrame()
for ind, row in df_SOTF_output_2.iterrows():
    location_id=row['location_id']
    store_location=(row['latitude_meas'],row['longitude_meas'])
    for zip_cd,v in zip_centers.items():
        dist=haversine(store_location,v,unit="mi")
        if dist<=10:
            # print(location_id,zip_cd)
            df=pd.DataFrame({"location_id":[location_id],"zip_cd":[zip_cd],"dist":[dist]})
            df_10_zips_all_new_stores=df_10_zips_all_new_stores.append(df)
df_10_zips_all_new_stores['zip_type']="zip_10"

In [17]:
df_new_store_zips=df_10_zips_all_new_stores[['location_id','zip_cd','zip_type']]
df_SOTF_output_2=pd.merge(df_SOTF_output_2,df_new_store_zips,how="left")
df_SOTF_output=df_SOTF_output_1.append(df_SOTF_output_2)
print(df_SOTF_output.shape,df_SOTF_output['location_id'].nunique(),df_SOTF_output['zip_cd'].nunique())

(2599, 9) 126 1691


In [18]:
dict_store_centers=df_SOTF_output[['location_id','latitude_meas','longitude_meas']].drop_duplicates()
dict_store_centers['store_centers']=dict_store_centers[['latitude_meas','longitude_meas']].values.tolist()
dict_store_centers=dict_store_centers.set_index("location_id").to_dict()['store_centers']

In [19]:
df_SOTF_output=df_SOTF_output.reset_index()
del df_SOTF_output['index']
df_SOTF_output['dist_miles']=np.nan

for i, row in df_SOTF_output.iterrows():
    store_num=row['location_id']
    store_center=dict_store_centers[store_num]
    zip_center=zip_centers[row['zip_cd']]
    dist=haversine(store_center,zip_center,unit="mi")
    df_SOTF_output.loc[i,"dist_miles"]=dist
    
    

In [20]:
df_unique_zip=df_SOTF_output[['zip_cd','location_id','dist_miles']]
df_unique_zip=df_unique_zip.sort_values(['zip_cd','dist_miles'],ascending=[True,True]).drop_duplicates("zip_cd")
# df_unique_zip=df_unique_zip[['zip_cd']]

In [21]:

df_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200301-134228-900.txt",
                           dtype=str,sep="|")
df_store_list['address']=df_store_list['address_line_1']+", "+df_store_list['city_nm']+", "+df_store_list['state_nm']+", "+df_store_list['zip_cd']
df_store_list=df_store_list[df_store_list['location_id'].isin(df_SOTF['location_id'].tolist())]

dict_qc_address_JL=df_store_list.set_index("location_id").to_dict()['address']


for k,v in dict_new_stores.items():
    dict_qc_address_JL.update({k:v['Google_Address']})
    
len(dict_qc_address_JL)

126

In [22]:
df_SOTF_output['QC_JL_Address']=df_SOTF_output['location_id'].apply(lambda x: dict_qc_address_JL[x])

In [23]:
writer=pd.ExcelWriter("./BL_zips_for_126_SOTF_stores_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_unique_zip.to_excel(writer,"unique_zips",index=False)
df_SOTF_output.to_excel(writer,"zips_all_SOTF",index=False)
writer.save()

In [24]:
df=df_SOTF_output.groupby("location_id")['City'].count().to_frame().reset_index()

In [25]:
df[df['City']==3]

,location_id,City
35,1765,3
114,5419,3


In [26]:
df_SOTF_output[df_SOTF_output['location_id']=="5419"]

,Reference #,City,Address,location_id,store_zip,latitude_meas,longitude_meas,zip_cd,zip_type,dist_miles,QC_JL_Address
2498,83,Galax,1083 East Stuart Drive,5419,24333,36.684598,-80.887184,24381,zip_10,4.043127,"1077 E Stuart Dr #100, Galax, VA 24333"
2499,83,Galax,1083 East Stuart Drive,5419,24333,36.684598,-80.887184,24330,zip_10,7.283085,"1077 E Stuart Dr #100, Galax, VA 24333"
2500,83,Galax,1083 East Stuart Drive,5419,24333,36.684598,-80.887184,24333,zip_10,2.958173,"1077 E Stuart Dr #100, Galax, VA 24333"
